In [1]:
import pandas as pd
import anal_preprocess as ap
import seaborn as sns
import matplotlib.pyplot as plt
import get_numcatg_data as gnca
pd.set_option('display.max_columns', None)
df = pd.read_csv('./diabetes.csv')
input_data = df.drop(columns=['Outcome'])

In [2]:
mdsource = gnca.GetNumericCategoryAllData(input_data, df)
col_out = "Outcome"
numeric_data, catg_data, ctg_data = mdsource.main(col_out)

ShapiroResult(statistic=0.6025138190827578, pvalue=1.2926899738528582e-38)
본 결측치 체크는 연속형 변수에 대하여 진행합니다.
결측치 데이터 컬럼 없음!
Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'],
      dtype='object')
이상치를 결측치로 대체한 후 KNN 방식으로 대체 ... 2차결측치 검증 진행
본 결측치 체크는 연속형 변수에 대하여 진행합니다.
결측치 데이터 컬럼 : 7개, ['Pregnancies', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'] 
Pregnancies                  4
Glucose                      0
BloodPressure                7
SkinThickness                1
Insulin                     34
BMI                          8
DiabetesPedigreeFunction    29
Age                          9
dtype: int64
KNN 알고리즘 대체
결측치 대체 후 검정 결측치 컬럼갯수 : 0 
본 결측치 체크는 연속형 변수에 대하여 진행합니다.
결측치 데이터 컬럼 없음!
범주형 데이터 없음


## 로지스틱 모델링

In [4]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import sklearn.svm as svm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np
import time
from imblearn.over_sampling import SMOTE

log = LogisticRegression()
xgb = XGBClassifier()
svm = svm.SVC(kernel='linear')

In [12]:
# 일반적인 로지스틱 모델링
kfold =KFold() # default = 5, 5개의 경우의 수로 분할하여 검증, ex) kfold = KFold(n_splits=3, shuffle=True) 
smote = SMOTE(random_state=0) # 클래스 불균형 해결안
class Model_class:
    def model_result(self, model, col_out, df_x, df_y): #model_df는 결과열을 포함하지 않는 데이터 프레임.
        pred_li = []
        for train_index, test_index in kfold.split(df_x):
            x_train, x_test = df_x.iloc[train_index, :], df_x.iloc[test_index, :]
            y_train, y_test = df_y.iloc[train_index, :], df_y.iloc[test_index, :]
            x_train_resample, y_train_resample = smote.fit_resample(x_train, y_train)
            
            start = time.time()
            model.fit(x_train_resample, y_train_resample)
            end = time.time()
            pred = model.predict(x_test)
            pred_li.append(accuracy_score(pred, y_test[col_out]))
        print(f"{end-start:.5f} sec")
        print(np.mean(pred_li))
        
    def pca_model_result(self, model, col_out, df_x, df_y):
        pred_li = []
        for train_index, test_index in kfold.split(df_x):
            x_train, x_test = df_x.iloc[train_index, :], df_x.iloc[test_index, :]
            y_train, y_test = df_y.iloc[train_index, :], df_y.iloc[test_index, :]
            x_train_resample, y_train_resample = smote.fit_resample(x_train, y_train)

            scaler = StandardScaler()
            x_train_res_s = scaler.fit_transform(x_train_resample)
            x_test_s = scaler.transform(x_test)
                                        
            num = len(df_x.columns)
            pca = PCA(n_components=num)
            x_train_pca = pca.fit_transform(x_train_res_s)
            x_test_pca = scaler.transform(x_test_s)

            start = time.time()
            model.fit(x_train_resample, y_train_resample)
            end = time.time()
            pred = model.predict(x_test)
            pred_li.append(accuracy_score(pred, y_test[col_out]))
        print(f"{end-start:.5f} sec")
        print(np.mean(pred_li))
            
mc = Model_class()
col_out = 'Outcome'
y = df[[col_out]]
'''
mc.model_result(log, col_out, numeric_data, y)
mc.model_result(xgb, col_out, numeric_data, y)
mc.model_result(svm, col_out, numeric_data, y)
'''
mc.pca_model_result(log, col_out, numeric_data, y)
mc.pca_model_result(xgb, col_out, numeric_data, y)
mc.pca_model_result(svm, col_out, numeric_data, y)

0.01421 sec
0.7527289703760293
0.04258 sec
0.7370087428910959
1.37605 sec
0.7448943213649096


In [9]:
# 속도개선
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA #PCA 주요 성분 분석방법
from sklearn.model_selection import train_test_split

smote = SMOTE(random_state = 0)
x_train, x_test, y_train, y_test = train_test_split(numeric_data, y, stratify=y, test_size=0.3, random_state=2022)
scaler = StandardScaler()
x_train_s = scaler.fit_transform(x_train)
num = len(numeric_data.columns)
pca = PCA(n_components=num)
x_train_pca = pca.fit(x_train_s)

print(pca.explained_variance_ratio_)
print(pca.explained_variance_ratio_[:5].sum())

[0.26638727 0.22566174 0.12075826 0.11202891 0.09170219 0.08598093
 0.04966146 0.04781923]
0.8165383766722168
